In [9]:
import numpy as np
import cv2
import csv

cap = cv2.VideoCapture('throwback_3_stand.mp4')
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
fgbg = cv2.createBackgroundSubtractorMOG2()

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('throwback_3_stand_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

with open('throwback_3_stand_csvtest.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['frame_counter', 'rectangle_counter','x','y','Width','Height','area'])

frame_counter = 0
while True:
    ret, frame = cap.read()
    rectangle_counter = 0
    if not ret:
        break

    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    contours, hierarchy = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        perimeter = cv2.arcLength(c, True)
        area = cv2.contourArea(c)
        if 50 < perimeter < 400 and area > 100:
            x, y, w, h = cv2.boundingRect(c)
            rectangle_counter += 1
            with open('throwback_3_stand_csvtest.csv', 'a', newline='') as csvfile:  # Use 'a' for append mode
                writer = csv.writer(csvfile)
                writer.writerow([frame_counter, rectangle_counter,x,y,w,h,area])
            cv2.putText(frame, str(rectangle_counter), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    out.write(frame)
    out.write(fgmask)

    frame_counter += 1

out.release()
cap.release()
cv2.destroyAllWindows()


In [4]:
import numpy as np
import cv2

cap = cv2.VideoCapture('court1.mp4')
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
fgbg = cv2.createBackgroundSubtractorMOG2()

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('output1.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

with open('rect_coordinates.txt', 'w') as f:
    #f.write("Frame\tRectangle\tX\tY\tWidth\tHeight\tPerimeter\tArea\n")
    f.write("Frame\tRectangle\t\tPerimeter\n")

frame_counter = 0

while True:
    ret, frame = cap.read()
    rectangle_counter = 0
    if not ret:
        break

    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    # 转换为灰度图像
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 应用阈值
    _, thresholded_frame = cv2.threshold(gray_frame, 130, 255, cv2.THRESH_BINARY)

    contours, hierarchy = cv2.findContours(thresholded_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        perimeter = cv2.arcLength(c, True)
        area = cv2.contourArea(c)
        if perimeter > 50 and perimeter < 400 and area > 100:  # 根据需要调整阈值
            x, y, w, h = cv2.boundingRect(c)
            rectangle_counter += 1
            # 将矩形框的坐标保存到文本文件中
            with open('rect_coordinates.txt', 'a') as f:
                f.write(f"{frame_counter}\t{rectangle_counter}\t\t{perimeter}\n")
            # 在视频帧上显示矩形框编号
            cv2.putText(frame, str(rectangle_counter), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # 保存帧到视频文件
    out.write(frame)
    out.write(fgmask)

    #cv2.imshow('frame', frame)
    #cv2.imshow('fgmask', fgmask)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    frame_counter += 1

out.release()
cap.release()
cv2.destroyAllWindows()


In [15]:
import numpy as np
import cv2

cap = cv2.VideoCapture('court1.mp4')
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
fgbg = cv2.createBackgroundSubtractorMOG2()

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('output1.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

with open('rect_coordinates.txt', 'w') as f:
    #f.write("Frame\tRectangle\tX\tY\tWidth\tHeight\tPerimeter\tArea\n")
    f.write("Frame\tRectangle\t\tPerimeter\n")

frame_counter = 0

while True:
    ret, frame = cap.read()
    rectangle_counter = 0
    if not ret:
        break

    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    lower_range = np.array([80, 80, 50])  # 设置低阈值，根据需要调整
    upper_range = np.array([200, 200, 180])  # 设置高阈值，根据需要调整

    filtered_frame = cv2.inRange(frame, lower_range, upper_range)

    contours, hierarchy = cv2.findContours(filtered_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    for c in contours:
        perimeter = cv2.arcLength(c, True)
        area = cv2.contourArea(c)
        if perimeter > 50 and perimeter < 400 and area > 100:  # 根据需要调整阈值
            x, y, w, h = cv2.boundingRect(c)
            rectangle_counter += 1
            # 将矩形框的坐标保存到文本文件中
            with open('rect_coordinates.txt', 'a') as f:
                #f.write(f"{frame_counter}\t{rectangle_counter}\t{x}\t{y}\t{w}\t{h}\t\t{perimeter}\t{area}\n")
                f.write(f"{frame_counter}\t{rectangle_counter}\t\t{perimeter}\n")
            # 在视频帧上显示矩形框编号
            cv2.putText(frame, str(rectangle_counter), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # 保存帧到视频文件
    out.write(frame)
    out.write(fgmask)

    #cv2.imshow('frame', frame)
    #cv2.imshow('fgmask', fgmask)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    frame_counter += 1

out.release()
cap.release()
cv2.destroyAllWindows()

In [29]:
import numpy as np
import cv2

cap = cv2.VideoCapture('court1.mp4')
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

# Customize the Background Subtractor MOG2
history = 500  # You can adjust this value based on your requirements
varThreshold = 16  # You can adjust this value based on your requirements
fgbg = cv2.createBackgroundSubtractorMOG2(history=history, varThreshold=varThreshold)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('output1.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

with open('rect_coordinates.txt', 'w') as f:
    #f.write("Frame\tRectangle\tX\tY\tWidth\tHeight\tPerimeter\tArea\n")
    f.write("Frame\tRectangle\t\tPerimeter\n")

frame_counter = 0
while True:
    ret, frame = cap.read()
    rectangle_counter = 0
    if not ret:
        break
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    contours, hierarchy = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        perimeter = cv2.arcLength(c, True)
        area = cv2.contourArea(c)
        if perimeter > 50 and perimeter < 400 and area > 100: 
            x, y, w, h = cv2.boundingRect(c)
            rectangle_counter += 1
            with open('rect_coordinates.txt', 'a') as f:
                f.write(f"{frame_counter}\t{rectangle_counter}\t\t{perimeter}\n")
            cv2.putText(frame, str(rectangle_counter), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    out.write(frame)
    out.write(fgmask)
    
    frame_counter += 1

out.release()
cap.release()
cv2.destroyAllWindows()

In [39]:
import csv

def extract_rows_by_frame(input_filename, output_filename, target_frame):
    with open(input_filename, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)  # Read the header row

        # Filter rows with the target frame
        filtered_rows = [row for row in reader if int(row[0]) == target_frame]

    # Write the filtered rows to a new CSV file
    with open(output_filename, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(header)
        writer.writerows(filtered_rows)

# Replace 'csvtest.csv' with the actual filename and 'x' with the desired frame number
extract_rows_by_frame('csvtest1.csv', 'x.csv', 17)